In [9]:
import pickle
from collections import defaultdict

with open("../checkpoints/association_rules.pkl", "rb") as f:
    assoc_data = pickle.load(f)

context_rule_index = assoc_data["context_rule_index"]

print("Loaded association rules OK")


Loaded association rules OK


In [10]:
def build_context_key(order_hour, is_weekend):
    time_bucket = (
        "morning" if 6 <= order_hour < 12 else
        "afternoon" if 12 <= order_hour < 18 else
        "evening"
    )

    day_type = "weekend" if is_weekend else "weekday"

    return f"time={time_bucket}|{day_type}"


In [11]:
def recommend_products(
    basket,
    context_key,
    context_rule_index,
    top_k=5,
    min_conf=0.05,
    min_lift=1.0
):
    scores = defaultdict(float)

    if context_key not in context_rule_index:
        return []

    context_rules = context_rule_index[context_key]

    for item in basket:
        item = str(item)   # ⚠ key là string "196"

        if item not in context_rules:
            continue

        for rule in context_rules[item]:
            if rule["confidence"] < min_conf:
                continue
            if rule["lift"] < min_lift:
                continue

            rec = rule["consequent"]
            scores[rec] += rule["score"]

    ranked = sorted(
        scores.items(),
        key=lambda x: x[1],
        reverse=True
    )

    return [item for item, _ in ranked[:top_k]]


In [12]:
example_basket = [196, 12427, 10258]

context_key = build_context_key(
    order_hour=8,
    is_weekend=False
)

recommend_products(
    basket=example_basket,
    context_key=context_key,
    context_rule_index=context_rule_index,
    top_k=5
)


[]